# SE446 - Week 4A: HiveQL Practice with Emulator

## 📚 Learning Objectives

By the end of this notebook, you will be able to:
1. Write DDL statements (CREATE TABLE, DROP TABLE, DESCRIBE)
2. Query data using SELECT, WHERE, GROUP BY, ORDER BY, LIMIT
3. Use aggregation functions and HAVING clauses
4. Perform JOINs across multiple tables
5. Apply window functions (RANK, ROW_NUMBER, running totals)
6. Read EXPLAIN output to understand query execution

---

## 1. Setup: The HiveQL Emulator

### ⚠️ Important Context: Emulation vs. Real Cluster

**Note:** Just like we used a Python emulator for MapReduce in Week 3, this notebook uses a **local SQLite-based emulator** to practice HiveQL.

**Why use an emulator?**
- Practice SQL/HiveQL syntax without needing cluster access
- Instant feedback — no waiting for YARN to schedule jobs
- Focus on **query logic**, not infrastructure

**What's the same?** The SQL syntax you learn here is nearly identical to what you'll run on the real Hive cluster in Notebook 4B.

**What's different?** Real Hive translates your SQL into MapReduce/Tez jobs that run across the cluster. Here, SQLite processes everything locally.

In [ ]:
import sqlite3
import pandas as pd
import random

# Create in-memory database (like Hive's Metastore + HDFS combined)
conn = sqlite3.connect(':memory:')

def hive_query(sql, show=True):
    """
    Execute a HiveQL-compatible query and display results.
    
    Parameters:
    - sql: SQL query string
    - show: if True, display results as DataFrame
    
    Returns: pandas DataFrame of results
    """
    try:
        cursor = conn.execute(sql)
        if cursor.description:  # SELECT queries return data
            columns = [desc[0] for desc in cursor.description]
            rows = cursor.fetchall()
            df = pd.DataFrame(rows, columns=columns)
            if show:
                display(df)
            return df
        else:  # DDL/DML queries (CREATE, INSERT, etc.)
            conn.commit()
            print(f"✅ Query executed successfully.")
            return None
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print("✅ HiveQL Emulator ready!")
print("   Use hive_query('YOUR SQL HERE') to run queries.")

## 2. Generate Datasets

We'll create three datasets to practice with:

| Dataset | Rows | Purpose |
|---------|------|---------|
| `chicago_crimes` | 1,000 | Crime analysis (DDL, aggregations) |
| `nyc_taxi` | 500 | Trip data (JOINs with weather) |
| `nyc_weather` | 365 | Weather conditions (JOIN target) |

In [ ]:
random.seed(42)

# ──────────────────────────────────────────────
# Dataset 1: Chicago Crimes (1,000 records)
# ──────────────────────────────────────────────
crime_types = ['THEFT', 'BATTERY', 'ASSAULT', 'CRIMINAL DAMAGE', 'BURGLARY', 'NARCOTICS']
crime_weights = [300, 250, 150, 120, 100, 80]
locations = ['STREET', 'RESIDENCE', 'APARTMENT', 'SIDEWALK', 'STORE', 'PARKING LOT']
districts = list(range(1, 9))

crimes_data = []
for i in range(1, 1001):
    ct = random.choices(crime_types, weights=crime_weights, k=1)[0]
    year = random.choice([2019, 2020, 2021, 2022, 2023, 2024])
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    crimes_data.append((
        i,
        f'HX{random.randint(100000,999999)}',
        f'{month:02d}/{day:02d}/{year}',
        ct,
        random.choice(locations),
        random.choice(districts),
        random.choice([True, False]),
    ))

conn.execute('''CREATE TABLE chicago_crimes (
    id INTEGER, case_number TEXT, date TEXT,
    primary_type TEXT, location_desc TEXT,
    district INTEGER, arrest BOOLEAN
)''')
conn.executemany('INSERT INTO chicago_crimes VALUES (?,?,?,?,?,?,?)', crimes_data)
conn.commit()

# ──────────────────────────────────────────────
# Dataset 2: NYC Taxi Trips (500 records)
# ──────────────────────────────────────────────
payment_types = ['Credit Card', 'Cash', 'No Charge', 'Dispute']
taxi_data = []
for i in range(1, 501):
    day_offset = random.randint(0, 364)
    month = (day_offset // 30) + 1
    if month > 12: month = 12
    day = (day_offset % 28) + 1
    pickup_date = f'2024-{month:02d}-{day:02d}'
    distance = round(random.uniform(0.5, 25.0), 1)
    fare = round(distance * 2.5 + random.uniform(2, 8), 2)
    tip = round(fare * random.uniform(0, 0.3), 2)
    taxi_data.append((
        i, pickup_date, distance, fare, tip,
        round(fare + tip, 2),
        random.choice(payment_types)
    ))

conn.execute('''CREATE TABLE nyc_taxi (
    trip_id INTEGER, pickup_date TEXT, distance REAL,
    fare REAL, tip REAL, total REAL, payment_type TEXT
)''')
conn.executemany('INSERT INTO nyc_taxi VALUES (?,?,?,?,?,?,?)', taxi_data)
conn.commit()

# ──────────────────────────────────────────────
# Dataset 3: NYC Weather (365 records)
# ──────────────────────────────────────────────
conditions = ['Clear', 'Cloudy', 'Rain', 'Snow', 'Fog']
weather_data = []
for day_offset in range(365):
    month = (day_offset // 30) + 1
    if month > 12: month = 12
    day = (day_offset % 28) + 1
    date = f'2024-{month:02d}-{day:02d}'
    # Seasonal temperature variation
    base_temp = 50 + 25 * (1 - abs(month - 7) / 6)
    temp_high = round(base_temp + random.uniform(-5, 5), 1)
    temp_low = round(temp_high - random.uniform(8, 18), 1)
    precip = round(random.uniform(0, 1.5), 2) if random.random() < 0.3 else 0.0
    cond = 'Rain' if precip > 0.5 else ('Snow' if precip > 0 and temp_high < 35 else random.choice(['Clear', 'Cloudy', 'Fog']))
    weather_data.append((date, temp_high, temp_low, precip, cond))

conn.execute('''CREATE TABLE nyc_weather (
    date TEXT, temp_high REAL, temp_low REAL,
    precipitation REAL, condition TEXT
)''')
conn.executemany('INSERT INTO nyc_weather VALUES (?,?,?,?,?)', weather_data)
conn.commit()

print("✅ All datasets loaded!")
print(f"   📊 chicago_crimes: {len(crimes_data):,} rows")
print(f"   📊 nyc_taxi:       {len(taxi_data):,} rows")
print(f"   📊 nyc_weather:    {len(weather_data):,} rows")

In [ ]:
# Quick preview of each table
print("── Chicago Crimes (first 5 rows) ──")
hive_query('SELECT * FROM chicago_crimes LIMIT 5')

print("\n── NYC Taxi (first 5 rows) ──")
hive_query('SELECT * FROM nyc_taxi LIMIT 5')

print("\n── NYC Weather (first 5 rows) ──")
hive_query('SELECT * FROM nyc_weather LIMIT 5')

---

## 3. Exercise 1: DDL — Creating and Inspecting Tables 🏗️

In Hive, you define tables using **DDL** (Data Definition Language) statements.

### Key HiveQL DDL Commands

| Command | Purpose |
|---------|--------|
| `CREATE TABLE` | Define a new table |
| `CREATE EXTERNAL TABLE` | Define table over existing HDFS data |
| `DROP TABLE` | Remove a table |
| `DESCRIBE table_name` | Show column names and types |
| `SHOW TABLES` | List all tables in current database |

In [ ]:
# SHOW TABLES — List all tables
# In Hive: SHOW TABLES;
# In our emulator (SQLite):
hive_query("SELECT name FROM sqlite_master WHERE type='table'")

In [ ]:
# DESCRIBE — Inspect table structure
# In Hive: DESCRIBE chicago_crimes;
# In our emulator (SQLite):
hive_query("PRAGMA table_info(chicago_crimes)")

In [ ]:
# CREATE TABLE — Let's create a new summary table
# In Hive this would be: CREATE TABLE crime_summary AS SELECT ...
# This is called CTAS (Create Table As Select)

hive_query('''
    CREATE TABLE crime_summary AS
    SELECT primary_type, COUNT(*) as total_crimes
    FROM chicago_crimes
    GROUP BY primary_type
''')

# Verify it was created
hive_query('SELECT * FROM crime_summary ORDER BY total_crimes DESC')

In [ ]:
# DROP TABLE — Remove a table
hive_query('DROP TABLE crime_summary')

# Verify it's gone
hive_query("SELECT name FROM sqlite_master WHERE type='table'")

### 🎯 Your Turn: DDL Practice

**Task**: Create a table called `theft_crimes` containing only THEFT records from `chicago_crimes`, then inspect it.

In [ ]:
# TODO: Create table theft_crimes using CTAS
# Hint: CREATE TABLE theft_crimes AS SELECT ... WHERE primary_type = 'THEFT'

# YOUR CODE HERE


# TODO: Count how many rows are in theft_crimes
# Hint: SELECT COUNT(*) ...

# YOUR CODE HERE


---

## 4. Exercise 2: Basic Queries — SELECT, WHERE, GROUP BY 📊

These are the bread-and-butter of HiveQL. If you know SQL, you already know this!

### Query Pattern
```sql
SELECT column1, AGG(column2)
FROM table_name
WHERE condition
GROUP BY column1
ORDER BY AGG(column2) DESC
LIMIT N;
```

In [ ]:
# Query 1: Count crimes by type (same as Milestone 1, Task 2!)
# In MapReduce this required a mapper + reducer + cluster job.
# In HiveQL, it's ONE line:

print("📊 Crime Type Distribution")
print("─" * 40)
hive_query('''
    SELECT primary_type, COUNT(*) AS cnt
    FROM chicago_crimes
    GROUP BY primary_type
    ORDER BY cnt DESC
''')

In [ ]:
# Query 2: Filter — only crimes with arrests
print("🚔 Crimes with Arrests by Type")
print("─" * 40)
hive_query('''
    SELECT primary_type, COUNT(*) AS arrest_count
    FROM chicago_crimes
    WHERE arrest = 1
    GROUP BY primary_type
    ORDER BY arrest_count DESC
''')

In [ ]:
# Query 3: Top 5 crime locations
print("📍 Top 5 Crime Locations")
print("─" * 40)
hive_query('''
    SELECT location_desc, COUNT(*) AS cnt
    FROM chicago_crimes
    GROUP BY location_desc
    ORDER BY cnt DESC
    LIMIT 5
''')

### 🎯 Your Turn: Basic Queries

**Task 1**: Write a query to count crimes per year. Extract the year from the `date` column using `SUBSTR(date, 7, 4)`.

**Task 2**: Write a query to find how many crimes occurred on the STREET in district 3.

In [ ]:
# TODO Task 1: Count crimes per year
# Hint: SELECT SUBSTR(date, 7, 4) AS year, COUNT(*) AS cnt
#       FROM chicago_crimes GROUP BY ... ORDER BY ...

# YOUR CODE HERE


In [ ]:
# TODO Task 2: Count STREET crimes in district 3
# Hint: Use WHERE with two conditions (AND)

# YOUR CODE HERE


---

## 5. Exercise 3: Aggregations + HAVING 📈

### WHERE vs HAVING

| Clause | Filters... | Timing |
|--------|-----------|--------|
| `WHERE` | Individual rows | **Before** grouping |
| `HAVING` | Groups | **After** grouping |

**Rule of thumb**: Use `WHERE` when filtering raw data, use `HAVING` when filtering aggregated results.

In [ ]:
# Find districts with MORE than 100 crimes
print("🏢 High-Crime Districts (> 100 crimes)")
print("─" * 40)
hive_query('''
    SELECT district, COUNT(*) AS total_crimes
    FROM chicago_crimes
    GROUP BY district
    HAVING COUNT(*) > 100
    ORDER BY total_crimes DESC
''')

In [ ]:
# Arrest rate by crime type — combining multiple aggregations
print("📈 Arrest Rate by Crime Type")
print("─" * 50)
hive_query('''
    SELECT 
        primary_type,
        COUNT(*) AS total,
        SUM(CASE WHEN arrest = 1 THEN 1 ELSE 0 END) AS arrests,
        ROUND(
            SUM(CASE WHEN arrest = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            1
        ) AS arrest_rate_pct
    FROM chicago_crimes
    GROUP BY primary_type
    ORDER BY arrest_rate_pct DESC
''')

### 🎯 Your Turn: Aggregations

**Task 1**: Find crime types that have an arrest rate **greater than 50%**. Use `HAVING`.

**Task 2**: For each year, calculate the total number of crimes AND the number of arrests. Order by year.

In [ ]:
# TODO Task 1: Crime types with arrest rate > 50%
# Hint: Use the arrest rate formula from above inside a HAVING clause

# YOUR CODE HERE


In [ ]:
# TODO Task 2: Per-year crime count and arrest count
# Hint: SELECT SUBSTR(date,7,4) AS year, COUNT(*), SUM(CASE WHEN arrest=1 ...) ...

# YOUR CODE HERE


---

## 6. Exercise 4: JOINs — Combining Tables 🔗

### JOIN Types

| Type | Returns |
|------|--------|
| `INNER JOIN` | Only rows that match in **both** tables |
| `LEFT JOIN` | All rows from left + matching from right |
| `RIGHT JOIN` | All rows from right + matching from left |
| `FULL OUTER JOIN` | All rows from both tables |

We'll join `nyc_taxi` with `nyc_weather` on the date to analyze how weather affects taxi trips.

In [ ]:
# JOIN: Taxi trips + Weather on the same day
print("🔗 Taxi Trips with Weather Data (sample)")
print("─" * 60)
hive_query('''
    SELECT 
        t.trip_id,
        t.pickup_date,
        t.fare,
        t.distance,
        w.condition,
        w.temp_high
    FROM nyc_taxi t
    INNER JOIN nyc_weather w ON t.pickup_date = w.date
    LIMIT 10
''')

In [ ]:
# Analytical JOIN: Average fare and distance by weather condition
print("🌤️ Average Taxi Metrics by Weather")
print("─" * 50)
hive_query('''
    SELECT 
        w.condition,
        COUNT(*) AS num_trips,
        ROUND(AVG(t.fare), 2) AS avg_fare,
        ROUND(AVG(t.distance), 2) AS avg_distance,
        ROUND(AVG(t.tip), 2) AS avg_tip
    FROM nyc_taxi t
    INNER JOIN nyc_weather w ON t.pickup_date = w.date
    GROUP BY w.condition
    ORDER BY num_trips DESC
''')

### 🎯 Your Turn: JOINs

**Task 1**: Find the average taxi fare on days when the temperature was above 70°F vs. below 40°F.

**Task 2**: Find the total number of taxi trips and total revenue for each payment type, but only on rainy days.

In [ ]:
# TODO Task 1: Average fare by temperature bracket
# Hint: JOIN taxi with weather, use CASE WHEN w.temp_high > 70 THEN 'Hot'
#       WHEN w.temp_high < 40 THEN 'Cold' ELSE 'Moderate' END AS temp_bracket

# YOUR CODE HERE


In [ ]:
# TODO Task 2: Payment type revenue on rainy days only
# Hint: JOIN + WHERE w.condition = 'Rain' + GROUP BY payment_type

# YOUR CODE HERE


---

## 7. Exercise 5: Window Functions 🪟

Window functions perform calculations **across a set of rows** related to the current row — without collapsing the rows like GROUP BY does.

### Syntax
```sql
FUNCTION() OVER (
    PARTITION BY column   -- groups to compute within
    ORDER BY column       -- ordering within each group
)
```

### Common Window Functions

| Function | Purpose |
|----------|--------|
| `ROW_NUMBER()` | Assign a unique sequential number |
| `RANK()` | Rank with gaps (ties get same rank) |
| `DENSE_RANK()` | Rank without gaps |
| `SUM() OVER()` | Running/cumulative total |
| `AVG() OVER()` | Running/moving average |

In [ ]:
# RANK: Rank crime types by frequency within each district
print("🏆 Top Crime Types per District")
print("─" * 50)
hive_query('''
    SELECT * FROM (
        SELECT 
            district,
            primary_type,
            COUNT(*) AS crime_count,
            RANK() OVER (
                PARTITION BY district 
                ORDER BY COUNT(*) DESC
            ) AS rnk
        FROM chicago_crimes
        GROUP BY district, primary_type
    )
    WHERE rnk <= 3
    ORDER BY district, rnk
''')

In [ ]:
# Running total: Cumulative fare by trip order
print("💰 Running Total of Taxi Fares (first 15 trips)")
print("─" * 50)
hive_query('''
    SELECT 
        trip_id,
        fare,
        ROUND(SUM(fare) OVER (ORDER BY trip_id), 2) AS cumulative_fare
    FROM nyc_taxi
    ORDER BY trip_id
    LIMIT 15
''')

### 🎯 Your Turn: Window Functions

**Task 1**: Use `ROW_NUMBER()` to assign a unique row number to each crime within each district, ordered by `id`. Show the first 2 rows per district.

**Task 2**: Calculate a running average of taxi fares using `AVG(fare) OVER (ORDER BY trip_id ROWS BETWEEN 4 PRECEDING AND CURRENT ROW)`. Show the first 20 rows.

In [ ]:
# TODO Task 1: ROW_NUMBER per district, show first 2 per district
# Hint: Use a subquery with ROW_NUMBER() OVER (PARTITION BY district ORDER BY id)
#       then filter WHERE row_num <= 2

# YOUR CODE HERE


In [ ]:
# TODO Task 2: Moving average of taxi fares (5-trip window)
# Hint: ROUND(AVG(fare) OVER (ORDER BY trip_id ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 2)

# YOUR CODE HERE


---

## 8. Exercise 6: EXPLAIN — Understanding Query Execution 🔍

In Hive, `EXPLAIN` shows you **how** the query will be executed (which MapReduce/Tez stages are created). In our emulator, SQLite's `EXPLAIN QUERY PLAN` gives similar insight.

### Why care about EXPLAIN?
- Understand if your query does a **full table scan** vs. using an **index**
- See if joins are efficient
- Diagnose slow queries

In [ ]:
# EXPLAIN a simple query
print("🔍 Execution Plan: COUNT crimes by type")
print("─" * 50)
hive_query('''
    EXPLAIN QUERY PLAN
    SELECT primary_type, COUNT(*)
    FROM chicago_crimes
    GROUP BY primary_type
''')

In [ ]:
# Create an index (simulates what Hive partitioning does)
hive_query('CREATE INDEX idx_district ON chicago_crimes(district)')

# EXPLAIN with index
print("\n🔍 Execution Plan: Query with INDEX (like partition pruning)")
print("─" * 50)
hive_query('''
    EXPLAIN QUERY PLAN
    SELECT primary_type, COUNT(*)
    FROM chicago_crimes
    WHERE district = 3
    GROUP BY primary_type
''')

In [ ]:
# EXPLAIN a JOIN query
print("🔍 Execution Plan: JOIN query")
print("─" * 50)
hive_query('''
    EXPLAIN QUERY PLAN
    SELECT w.condition, AVG(t.fare)
    FROM nyc_taxi t
    JOIN nyc_weather w ON t.pickup_date = w.date
    GROUP BY w.condition
''')

---

## 9. 🏆 Challenge: Full Analytics Pipeline

Put it all together! Write a **single query** that answers:

> **"For each district, what is the #1 most common crime type, how many times did it occur, and what is its arrest rate?"**

This requires combining: GROUP BY, window functions (RANK), CASE WHEN, and a subquery.

In [ ]:
# TODO: Challenge — Top crime type per district with arrest rate
# Expected output columns: district, primary_type, crime_count, arrest_rate_pct
#
# Approach:
# 1. Inner query: GROUP BY district, primary_type to get counts + arrest rates
# 2. Add RANK() OVER (PARTITION BY district ORDER BY count DESC) 
# 3. Outer query: filter WHERE rnk = 1

# YOUR CODE HERE


---

## 10. HiveQL Cheat Sheet 📋

### DDL (Data Definition)

| Command | Example |
|---------|--------|
| Create table | `CREATE TABLE t (col1 INT, col2 STRING)` |
| Create external | `CREATE EXTERNAL TABLE t (...) LOCATION '/path'` |
| CTAS | `CREATE TABLE t AS SELECT ...` |
| Drop table | `DROP TABLE t` |
| Describe | `DESCRIBE t` |

### DML (Queries)

| Pattern | Example |
|---------|--------|
| Basic select | `SELECT col FROM t WHERE cond` |
| Aggregation | `SELECT col, COUNT(*) FROM t GROUP BY col` |
| HAVING | `... GROUP BY col HAVING COUNT(*) > 10` |
| JOIN | `SELECT ... FROM t1 JOIN t2 ON t1.key = t2.key` |
| Window | `RANK() OVER (PARTITION BY col ORDER BY col2)` |
| Subquery | `SELECT * FROM (SELECT ...) WHERE rnk = 1` |

### Common Functions

| Category | Functions |
|----------|----------|
| Aggregate | `COUNT`, `SUM`, `AVG`, `MIN`, `MAX` |
| String | `UPPER`, `LOWER`, `SUBSTR`, `CONCAT`, `LENGTH` |
| Conditional | `CASE WHEN ... THEN ... ELSE ... END` |
| Rounding | `ROUND(value, decimals)` |
| Window | `ROW_NUMBER`, `RANK`, `DENSE_RANK`, `LAG`, `LEAD` |

## 11. 🏠 Preparation for Notebook 4B

In the next notebook, you'll run **real HiveQL queries on the Hadoop cluster**.

**Before class:**
1. Make sure you can SSH into the cluster
2. Review the `CREATE EXTERNAL TABLE` syntax — you'll create a table over `/data/chicago_crimes.csv`
3. Think about this: the same crime queries you wrote here will now run as distributed Tez/MapReduce jobs!